In [ ]:
pip install optuna

In [1]:
# =======================================================
# Colab: LightGBM 7일 수요예측 (Optuna 최적화 포함) - ONE CELL
# =======================================================

import os
import json
import gc
import numpy as np
import pandas as pd
import optuna
from lightgbm import LGBMRegressor, early_stopping
import joblib

# 2. 시드 고정 및 기본 설정
SEED = 42
np.random.seed(SEED)

# =======================================================
# 3. 공통 함수 (피처 생성 및 평가)
# =======================================================

# 피처 엔지니어링 파라미터
LAGS  = [1, 2, 3, 7, 14, 21, 28]
ROLLS = [7, 14, 28]

def smape(y_true, y_pred) -> float:
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    denom = (np.abs(y_true) + np.abs(y_pred)) / 2.0
    diff  = np.abs(y_true - y_pred)
    denom[denom == 0] = 1.0
    return float(np.mean(diff / denom) * 100.0)

def build_train_table(df: pd.DataFrame) -> pd.DataFrame:
    df = df.sort_values(['영업장명_메뉴명','영업일자']).reset_index(drop=True)
    df['item_le'] = df['영업장명_메뉴명'].astype('category').cat.codes

    # Lags
    for k in LAGS:
        df[f'lag_{k}'] = df.groupby('영업장명_메뉴명')['매출수량'].shift(k)

    # Rolling(누수 방지: 1일 시프트 후)
    df['qty_shift1'] = df.groupby('영업장명_메뉴명')['매출수량'].shift(1)
    for w in ROLLS:
        df[f'roll_mean_{w}'] = (
            df.groupby('영업장명_메뉴명')['qty_shift1']
              .rolling(window=w, min_periods=w).mean()
              .reset_index(level=0, drop=True)
        )
        df[f'roll_std_{w}']  = (
            df.groupby('영업장명_메뉴명')['qty_shift1']
              .rolling(window=w, min_periods=w).std()
              .reset_index(level=0, drop=True)
        )

    df['roll_min_7'] = (
        df.groupby('영업장명_메뉴명')['qty_shift1']
          .rolling(window=7, min_periods=7).min()
          .reset_index(level=0, drop=True)
    )
    df['roll_max_7'] = (
        df.groupby('영업장명_메뉴명')['qty_shift1']
          .rolling(window=7, min_periods=7).max()
          .reset_index(level=0, drop=True)
    )
    df['roll_sum_7'] = (
        df.groupby('영업장명_메뉴명')['qty_shift1']
          .rolling(window=7, min_periods=7).sum()
          .reset_index(level=0, drop=True)
    )

    df['is_zero'] = (df['qty_shift1'] == 0).astype(float)
    df['zero_ratio_28'] = (
        df.groupby('영업장명_메뉴명')['is_zero']
          .rolling(window=28, min_periods=28).mean()
          .reset_index(level=0, drop=True)
    )

    # 캘린더(참조일)
    df['ref_year']       = df['영업일자'].dt.year
    df['ref_month']      = df['영업일자'].dt.month
    df['ref_day']        = df['영업일자'].dt.day
    df['ref_weekday']    = df['영업일자'].dt.weekday
    df['ref_is_weekend'] = df['ref_weekday'].isin([5,6]).astype(int)

    # 타깃(1~7일 후)
    for h in range(1, 8):
        df[f'y_h{h}'] = df.groupby('영업장명_메뉴명')['매출수량'].shift(-h)

    base_cols = (
        ['영업장명_메뉴명','item_le','영업일자',
         'ref_year','ref_month','ref_day','ref_weekday','ref_is_weekend']
        + [f'lag_{k}' for k in LAGS]
        + [f'roll_mean_{w}' for w in ROLLS]
        + [f'roll_std_{w}'  for w in ROLLS]
        + ['roll_min_7','roll_max_7','roll_sum_7','zero_ratio_28']
    )

    # long-format: h=1..7
    records = []
    for h in range(1, 8):
        tmp = df[base_cols + [f'y_h{h}']].copy()
        tmp = tmp.rename(columns={f'y_h{h}':'y'})
        tmp['h'] = h
        tmp['target_weekday']    = (df['영업일자'] + pd.to_timedelta(h, unit='D')).dt.weekday
        tmp['target_is_weekend'] = tmp['target_weekday'].isin([5,6]).astype(int)
        records.append(tmp)

    Xy = pd.concat(records, axis=0, ignore_index=True)
    Xy = Xy.rename(columns={'영업일자':'ref_date'})
    Xy = Xy.dropna()
    return Xy

def build_test_features(test_df: pd.DataFrame) -> pd.DataFrame:
    test_df = test_df.sort_values(['영업장명_메뉴명','영업일자']).reset_index(drop=True)
    test_df['item_le'] = test_df['영업장명_메뉴명'].astype('category').cat.codes

    feats = []
    for item, g in test_df.groupby('영업장명_메뉴명'):
        g = g.sort_values('영업일자')
        if len(g) < 28:
            continue
        last28 = g.tail(28).copy()
        ref_date = last28['영업일자'].iloc[-1]
        qty = last28['매출수량'].values.astype(float)

        base = {
            'item_le': int(last28['item_le'].iloc[-1]),
            'ref_year': ref_date.year,
            'ref_month': ref_date.month,
            'ref_day': ref_date.day,
            'ref_weekday': ref_date.weekday(),
            'ref_is_weekend': 1 if ref_date.weekday() in (5,6) else 0,
        }
        for k in LAGS:
            base[f'lag_{k}'] = float(qty[-k])
        for w in ROLLS:
            tail = qty[-w:]
            base[f'roll_mean_{w}'] = float(np.mean(tail))
            base[f'roll_std_{w}']  = float(np.std(tail, ddof=0))

        base['roll_min_7'] = float(np.min(qty[-7:]))
        base['roll_max_7'] = float(np.max(qty[-7:]))
        base['roll_sum_7'] = float(np.sum(qty[-7:]))
        base['zero_ratio_28'] = float(np.mean(qty == 0))

        for h in range(1, 8):
            row = base.copy()
            t_wd = (ref_date + pd.Timedelta(days=h)).weekday()
            row['h'] = h
            row['target_weekday'] = t_wd
            row['target_is_weekend'] = 1 if t_wd in (5,6) else 0
            row['영업장명_메뉴명'] = item
            feats.append(row)

    return pd.DataFrame(feats)


def calculate_weighted_smape(y_true: np.ndarray, y_pred: np.ndarray, df_info: pd.DataFrame, weights: dict = None) -> float:
    temp_df = df_info.copy()
    temp_df['actual'] = y_true
    temp_df['prediction'] = np.maximum(0, y_pred)

    total_score = 0
    if weights is None:
        unique_stores = temp_df['store'].unique()
        weights = {s: 1.0 for s in unique_stores}

    for s in temp_df['store'].unique():
        store_df = temp_df[temp_df['store'] == s]
        item_smapes = []
        for i in store_df['item'].unique():
            item_df = store_df[store_df['item'] == i]
            valid_days_df = item_df[item_df['actual'] != 0].copy()
            if len(valid_days_df) == 0:
                item_smapes.append(0.0)
                continue
            diff = np.abs(valid_days_df['actual'] - valid_days_df['prediction'])
            denom = np.abs(valid_days_df['actual']) + np.abs(valid_days_df['prediction'])
            smape_i = np.mean(2 * diff / denom)
            item_smapes.append(smape_i)
        store_smape = np.mean(item_smapes)
        ws = weights.get(s, 1.0)
        total_score += ws * store_smape
    return total_score


# =======================================================
# 4. 데이터 로드 및 전처리
# =======================================================
# 파일 업로드 확인
required_files = ["train.csv", "sample_submission.csv"] + [f"TEST_{i:02d}.csv" for i in range(10)]
for f in required_files:
    if not os.path.exists(f):
        raise FileNotFoundError(f"필수 파일이 없습니다: {f}. Colab에 파일을 업로드해주세요.")

train = pd.read_csv("train.csv", parse_dates=["영업일자"])
train = train.sort_values(['영업장명_메뉴명','영업일자'])
print("Original train shape:", train.shape, "| Unique items:", train['영업장명_메뉴명'].nunique())

Xy = build_train_table(train)
print("Built train table shape:", Xy.shape)

# =======================================================
# 5. 가중치 설계
# =======================================================
Xy['w'] = 1.0
rd = Xy['ref_date']

# (A) 기간별 버킷 가중치
Xy.loc[(rd >= '2023-01-01') & (rd <= '2023-12-31'), 'w'] = 0.9
Xy.loc[(rd >= '2023-06-01') & (rd <= '2023-07-15'), 'w'] = 1.3 # 덮어쓰기
Xy.loc[(rd >= '2024-01-01') & (rd <= '2024-04-30'), 'w'] = 1.0
Xy.loc[(rd >= '2024-05-01') & (rd <= '2024-05-31'), 'w'] = 1.2
Xy.loc[(rd >= '2024-06-01') & (rd <= '2024-06-15'), 'w'] = 1.4

# (B) 동적 가중치: 시간 감쇠, 주말, 예측 시점
HALF_LIFE_DAYS = 60
days_from_max = (Xy['ref_date'].max() - rd).dt.days.clip(lower=0)
decay = 0.5 ** (days_from_max / HALF_LIFE_DAYS)
Xy['w'] *= decay
Xy.loc[Xy['target_is_weekend'] == 1, 'w'] *= 1.10
Xy['w'] *= (1.0 + (8 - Xy['h']) * 0.02)

# =======================================================
# 6. Optuna를 이용한 하이퍼파라미터 최적화
# =======================================================
VALID_DAYS      = 14
N_ESTIMATORS    = 2000
# Optuna 실행 시에는 early stopping을 짧게 주어 탐색 속도를 높이는 것이 효율적
OPTUNA_EARLY_STOPPING = 50

# 데이터 분할
cutoff = Xy['ref_date'].max() - pd.Timedelta(days=VALID_DAYS)
train_mask = Xy['ref_date'] <= cutoff
valid_mask = Xy['ref_date'] >  cutoff

feature_cols = [c for c in Xy.columns if c not in ['y','영업장명_메뉴명','ref_date','w']]
X_train, y_train = Xy.loc[train_mask, feature_cols], Xy.loc[train_mask, 'y']
X_valid, y_valid = Xy.loc[valid_mask, feature_cols], Xy.loc[valid_mask, 'y']
w_train = Xy.loc[train_mask, 'w'].values
w_valid = Xy.loc[valid_mask, 'w'].values

# 평가 함수에 필요한 정보 미리 정의
info_df = Xy.loc[valid_mask, ['영업장명_메뉴명']].reset_index(drop=True)
info_df[['store', 'item']] = info_df['영업장명_메뉴명'].str.split('_', expand=True)

def objective(trial):
    """Optuna가 최적의 파라미터를 찾기 위해 반복적으로 호출할 함수"""
    params = {
        'objective': 'regression_l1',
        'n_estimators': N_ESTIMATORS,
        'random_state': SEED,
        'verbose': -1,
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 20, 150),
        'subsample': trial.suggest_float('subsample', 0.6, 0.95),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.6, 0.95),
        'min_child_samples': trial.suggest_int('min_child_samples', 20, 100),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
    }

    lgbm = LGBMRegressor(**params)
    callbacks = [early_stopping(stopping_rounds=OPTUNA_EARLY_STOPPING, verbose=False)]

    lgbm.fit(
        X_train, y_train,
        sample_weight=w_train,
        eval_set=[(X_valid, y_valid)],
        eval_sample_weight=[w_valid],
        eval_metric='l1',
        callbacks=callbacks
    )

    valid_pred = lgbm.predict(X_valid)
    score = calculate_weighted_smape(y_valid.values, valid_pred, info_df)
    return score

# Optuna 스터디 생성 및 최적화 실행
print("\nStarting Optuna hyperparameter optimization...")
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50, show_progress_bar=True)

# 최적화 결과 확인
print("\n" + "="*50)
print("Optuna Optimization Finished!")
print("Best trial's Weighted SMAPE:", study.best_value)
print("Best hyperparameters:", study.best_params)
print("="*50 + "\n")


# =======================================================
# 7. 최종 모델 학습
# =======================================================
print("Training the final model with the best hyperparameters...")
best_params = study.best_params
best_params['n_estimators'] = N_ESTIMATORS
best_params['random_state'] = SEED

final_model = LGBMRegressor(**best_params)

# 최종 모델 학습 시에는 Early Stopping을 더 길게 설정
FINAL_EARLY_STOPPING = 200
final_callbacks = [early_stopping(stopping_rounds=FINAL_EARLY_STOPPING, verbose=False)]
final_model.fit(
    X_train, y_train,
    sample_weight=w_train,
    eval_set=[(X_valid, y_valid)],
    eval_sample_weight=[w_valid],
    eval_metric='l1',
    callbacks=final_callbacks
)

# 최종 모델로 검증 데이터 예측 및 평가
valid_pred = final_model.predict(X_valid)
score_original = smape(y_valid.values, valid_pred)
score_weighted = calculate_weighted_smape(y_valid.values, valid_pred, info_df)

print(f"Final Valid SMAPE (Original): {score_original:.4f}")
print(f"Final Valid SMAPE (Competition Weighted): {score_weighted:.6f}\n")


# =======================================================
# 8. 모델 저장 및 추론/제출
# =======================================================
# 모델 및 피처 정보 저장
os.makedirs("lgbm_models", exist_ok=True)
joblib.dump(final_model, "lgbm_models/lgbm_weighted_optuna.pkl")
with open("lgbm_models/feature_cols.json", "w", encoding="utf-8") as f:
    json.dump({"feature_cols": feature_cols}, f, ensure_ascii=False, indent=2)
print("Final model and feature list saved.")

# 추론 및 제출 파일 생성
print("Generating submission file...")
sample = pd.read_csv("sample_submission.csv")
all_items = list(sample.columns[1:])
test_paths = [f"TEST_{i:02d}.csv" for i in range(10)]

all_rows = []
for i, tp in enumerate(test_paths):
    tdf = pd.read_csv(tp, parse_dates=['영업일자'])
    feats = build_test_features(tdf)

    if not feats.empty:
      # 최종 모델(final_model)을 사용하여 예측
      preds = final_model.predict(feats[feature_cols])
      feats['pred'] = np.maximum(0.0, preds)

      for h in range(1, 8):
          tag = f"TEST_{i:02d}+{h}일"
          row = {'영업일자': tag}
          sub = feats[feats['h']==h][['영업장명_메뉴명','pred']].set_index('영업장명_메뉴명')['pred']
          for item in all_items:
              row[item] = float(sub.get(item, 0.0))
          all_rows.append(row)

submission = pd.DataFrame(all_rows, columns=['영업일자'] + all_items)
submission.to_csv("submission_optuna.csv", index=False)
print("\nSubmission file created successfully -> submission_optuna.csv")
display(submission.head())

c:\Users\syyun\Desktop\Programming\Hackerton\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Original train shape: (102676, 3) | Unique items: 193
Built train table shape: (675500, 29)


[I 2025-08-11 10:42:03,266] A new study created in memory with name: no-name-21aca522-1a6b-4ebc-b67e-a55b18fee6e6



Starting Optuna hyperparameter optimization...


Best trial: 0. Best value: 7.96215:   2%|▏         | 1/50 [04:47<3:55:09, 287.96s/it]

...

Best trial: 31. Best value: 7.61356:  98%|█████████▊| 49/50 [2:58:22<03:49, 229.28s/it]

[I 2025-08-11 13:40:25,564] Trial 48 finished with value: 8.09454791176935 and parameters: {'learning_rate': 0.09652930056017227, 'num_leaves': 41, 'subsample': 0.654215733025122, 'feature_fraction': 0.9194089062674167, 'min_child_samples': 97, 'lambda_l1': 0.24070876483290632, 'lambda_l2': 8.155771548456004}. Best is trial 31 with value: 7.613556290419443.


Best trial: 31. Best value: 7.61356: 100%|██████████| 50/50 [3:06:32<00:00, 223.85s/it]


[I 2025-08-11 13:48:35,788] Trial 49 finished with value: 7.753789190179663 and parameters: {'learning_rate': 0.010347704964471226, 'num_leaves': 132, 'subsample': 0.7294567249657544, 'feature_fraction': 0.780202247407111, 'min_child_samples': 86, 'lambda_l1': 0.05546435929752246, 'lambda_l2': 0.43711410325308503}. Best is trial 31 with value: 7.613556290419443.

Optuna Optimization Finished!
Best trial's Weighted SMAPE: 7.613556290419443
Best hyperparameters: {'learning_rate': 0.02223107357910896, 'num_leaves': 143, 'subsample': 0.6997533428396597, 'feature_fraction': 0.7656843584182942, 'min_child_samples': 100, 'lambda_l1': 6.7614576606843055, 'lambda_l2': 5.4694159958898325}

Training the final model with the best hyperparameters...
Final Valid SMAPE (Original): 132.6214
Final Valid SMAPE (Competition Weighted): 5.819903

Final model and feature list saved.
Generating submission file...

Submission file created successfully -> submission_optuna.csv


,영업일자,느티나무 셀프BBQ_1인 수저세트,느티나무 셀프BBQ_BBQ55(단체),"느티나무 셀프BBQ_대여료 30,000원","느티나무 셀프BBQ_대여료 60,000원","느티나무 셀프BBQ_대여료 90,000원","느티나무 셀프BBQ_본삼겹 (단품,실내)",느티나무 셀프BBQ_스프라이트 (단체),느티나무 셀프BBQ_신라면,느티나무 셀프BBQ_쌈야채세트,...,화담숲주막_스프라이트,화담숲주막_참살이 막걸리,화담숲주막_찹쌀식혜,화담숲주막_콜라,화담숲주막_해물파전,화담숲카페_메밀미숫가루,화담숲카페_아메리카노 HOT,화담숲카페_아메리카노 ICE,화담숲카페_카페라떼 ICE,화담숲카페_현미뻥스크림
0,TEST_00+1일,5.614698,7.051601,4.215854,3.712702,0.489786,1.200534,1.642430,2.775638,2.430695,...,8.215223,21.438593,19.012745,9.388106,44.124607,37.022834,6.166973,30.672032,9.355602,18.145919
1,TEST_00+2일,2.921619,4.610685,1.985845,1.726812,0.089844,0.487418,3.505720,1.508797,1.167092,...,0.266943,2.360555,2.028825,0.000000,1.606676,2.513820,0.000000,2.608939,0.000000,0.009193
2,TEST_00+3일,3.764345,10.295247,2.431092,2.030662,0.456175,0.851525,5.819088,1.789391,1.489997,...,0.973836,7.934783,7.552458,0.892002,23.717235,20.311741,2.693818,26.085880,5.523975,11.159912
3,TEST_00+4일,4.010242,15.392397,2.409802,2.097987,0.477158,0.870695,5.436389,1.808561,1.533672,...,1.122577,7.895230,7.660190,0.958656,21.939712,15.248174,1.543787,18.847910,3.915985,9.080520
4,TEST_00+5일,6.695717,49.871573,3.765098,3.147017,0.788555,1.238716,12.153636,2.485572,1.998570,...,2.388276,10.737758,10.275491,2.459643,28.021674,22.077062,2.674417,25.186979,5.926742,11.008191
